<a href="https://colab.research.google.com/github/cemreefe/cmpe493-project/blob/main/huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install xmltodict

import os
import io   
import re
import json
import math
import pickle
import string
import tarfile
import xmltodict
import numpy as np
import pandas as pd

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def read_file(path):
  with open(path, 'r') as f:
    return f.read()

In [5]:
if not os.path.exists('drive/MyDrive/CMPE/CMPE493'):
  os.makedirs('drive/MyDrive/CMPE/CMPE493')

In [6]:
if not os.path.exists('drive/MyDrive/CMPE/CMPE493/topics-rnd5.xml'):
  !curl https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml --output drive/MyDrive/CMPE/CMPE493/topics-rnd5.xml

if not os.path.exists('drive/MyDrive/CMPE/CMPE493/qrels-covid_d5_j0.5-5.txt'):
  !curl https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt --output drive/MyDrive/CMPE/CMPE493/qrels-covid_d5_j0.5-5.txt

if not os.path.exists('drive/MyDrive/CMPE/CMPE493/cord-19_2020-07-16.tar.gz'):
  !curl https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-07-16.tar.gz --output drive/MyDrive/CMPE/CMPE493/cord-19_2020-07-16.tar.gz

In [7]:
if not os.path.exists('2020-07-16'):
  tar = tarfile.open('drive/MyDrive/CMPE/CMPE493/cord-19_2020-07-16.tar.gz', "r:gz")
  tar.extractall()
  tar.close()

In [8]:
df_metadata = pd.read_csv('2020-07-16/metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
del df_metadata['sha'], df_metadata['source_x'], df_metadata['doi'], df_metadata['pmcid'], df_metadata['pubmed_id'], df_metadata['license'], df_metadata['publish_time'], df_metadata['authors'], df_metadata['journal'], df_metadata['mag_id'], df_metadata['who_covidence_id'], df_metadata['arxiv_id'], df_metadata['pdf_json_files'], df_metadata['pmc_json_files'], df_metadata['url'], df_metadata['s2_id']

In [10]:
df_metadata.drop_duplicates(subset='cord_uid', keep='first', inplace=True)

In [11]:
df_metadata

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...
...,...,...,...
192504,z4ro6lmh,Rapid radiological improvement of COVID-19 pne...,NaN
192505,hi8k8wvb,SARS E protein in phospholipid bilayers: an an...,Abstract We report on an anomalous X-ray refle...
192506,ma3ndg41,Italian Society of Interventional Cardiology (...,COVID‐19 pandemic raised the issue to guarante...
192507,wh10285j,"Nimble, Together: A Training Program's Respons...",NaN


In [12]:
topic_relevances = 'topic iter document_id judgement\n' + read_file('drive/MyDrive/CMPE/CMPE493/qrels-covid_d5_j0.5-5.txt')

df_relevances = pd.read_csv(  io.StringIO(topic_relevances)  , sep=" ")
del df_relevances['iter']

df_relevances

,topic,document_id,judgement
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,010vptx3,2
3,1,0194oljo,1
4,1,021q9884,1
...,...,...,...
69313,50,zvop8bxh,2
69314,50,zwf26o63,1
69315,50,zwsvlnwe,0
69316,50,zxr01yln,1


In [13]:
topics_obj = xmltodict.parse(read_file('drive/MyDrive/CMPE/CMPE493/topics-rnd5.xml'))
topics     = json.loads(json.dumps(topics_obj))

topics_dict = {}
for topic in topics['topics']['topic']:
  # a topic has the following fields:
  #  * @number
  #  * narrative
  #  * query
  #  * question
  topics_dict[topic['@number']] = topic['query'] + ' ' + topic['question'] + ' ' + topic['narrative']

In [14]:
len(topics_dict)

50

# Data so far

* `topics_dict` 
      has `topic-id` for keys, and topic description for values
* `df_relevances` 
      has the following three columns:
      topic	document-id	judgement
* `df_metadata`
      holds information about the documents
      has the following three columns (others are deleted):
      cord_uid	title	abstract


In [15]:
nltk.download('stopwords')
#stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
docs = np.array(df_metadata)
contents = {}

for doc in docs:
  contents[doc[0]] = f'{doc[1]} {doc[2]}'

### `contents` is a dictionary with document id keys and f'{document title} {document content}' values.
```
document_id: f'{document_title} {document_content}'
```

In [ ]:
!pip install -U sentence-transformers

In [18]:
!unzip trec_eval-master.zip

unzip:  cannot find or open trec_eval-master.zip, trec_eval-master.zip.zip or trec_eval-master.zip.ZIP.


In [58]:
porter_stemmer = PorterStemmer()
sw = stopwords.words('english')

# preprocessing
# case folding
# punctuation removal
# number deletion
# stemming & stopword removal
def preprocess(s):
  s = s.casefold()
  s = s.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
  s = re.sub("\d+", "", s) #Delete numbers
  s = ' '.join([porter_stemmer.stem(word) for word in s.split() if word not in sw and word != 'nan'])
  return s

https://huggingface.co/sentence-transformers/ce-ms-marco-TinyBERT-L-2

In [59]:
for key, value in contents.items():
  contents[key] = preprocess(value)

In [70]:
from sentence_transformers import CrossEncoder
import transformers
model_name = 'sentence-transformers/ce-ms-marco-TinyBERT-L-6'
model = CrossEncoder(model_name, max_length=512)
model.tokenizer = transformers.BertTokenizerFast.from_pretrained(model_name)

In [71]:
pairs = df_relevances[['topic', 'document_id']]
pairs = np.array(pairs)
pairs

array([[1, '005b2j4b'],
       [1, '00fmeepz'],
       [1, '010vptx3'],
       ...,
       [50, 'zwsvlnwe'],
       [50, 'zxr01yln'],
       [50, 'zz8wvos9']], dtype=object)

In [72]:
value_pairs = [[]]*len(pairs)

In [73]:
for i,pair in enumerate(pairs):
  value_pairs[i] = (topics_dict[str(pair[0])], contents[pair[1]])

In [74]:
len(value_pairs)

69318

In [75]:
%%time
scores = model.predict(value_pairs)

CPU times: user 6min 20s, sys: 3min 54s, total: 10min 15s
Wall time: 9min 42s


In [76]:
scores

array([0.00980095, 0.05991739, 0.0151718 , ..., 0.00376705, 0.00209249,
       0.0374207 ], dtype=float32)

In [77]:
results = []
for score, pair in zip(scores, pairs):
  results.append(f'{pair[0]} 0 {pair[1]} 0 {score} 0')

In [78]:
results[:10]

['1 0 005b2j4b 0 0.00980095099657774 0',
 '1 0 00fmeepz 0 0.0599173903465271 0',
 '1 0 010vptx3 0 0.015171797946095467 0',
 '1 0 0194oljo 0 0.008037539198994637 0',
 '1 0 021q9884 0 0.00019172504835296422 0',
 '1 0 02f0opkr 0 0.058555226773023605 0',
 '1 0 047xpt2c 0 0.009575512260198593 0',
 '1 0 04ftw7k9 0 0.00019838951993733644 0',
 '1 0 05qglt1f 0 0.002087506465613842 0',
 '1 0 05vx82oo 0 0.0002874651690945029 0']

In [79]:
# Write results to file
with open('results_preprocessed_tinybertl6', 'w') as f:
    f.write('\n'.join(results))

In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:18<00:00, 21.3MB/s]


In [21]:
list(contents.items())[:10]

[('ug7v899j',
  'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with 

In [29]:
len(df_relevances.document_id.unique())

37924

In [ ]:
%%time

def get_doc_vectors():
  return model.encode(list(contents.keys()))

doc_vectors = get_doc_vectors()

In [30]:
%%time

def get_topic_vectors():
  return model.encode(list(topics_dict.values()))

topic_vectors = get_topic_vectors()

CPU times: user 12.2 s, sys: 6.65 ms, total: 12.2 s
Wall time: 12.3 s


In [31]:
topic_vectors.shape

(50, 768)

In [46]:
len(topic_vectors)

50

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

sims = cosine_similarity(doc_vectors, topic_vectors)
sims.shape

(191175, 50)

In [48]:
all_sims = pd.DataFrame(sims, columns=list(np.array(range(50))+1))
all_sims.insert(loc=0, column='doc_id', value=contents.keys())
all_sims

,doc_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,ug7v899j,0.790289,0.560938,0.513738,0.783259,0.620462,0.490478,0.798114,0.823216,0.636358,0.835604,0.770633,0.769841,0.415509,0.803367,0.641954,0.446103,0.608661,0.677222,0.796022,0.743759,0.755667,0.694395,0.694711,0.816873,0.663551,0.635219,0.422218,0.556565,0.815236,0.729534,0.773056,0.795988,0.792966,0.788263,0.722316,0.685445,0.701736,0.805388,0.681546,0.812574,0.835786,0.743074,0.516458,0.687099,0.759748,0.785593,0.805212,0.739708,0.646190,0.090384
1,02tnwd4m,0.877255,0.748425,0.515580,0.826949,0.709421,0.599424,0.865828,0.861213,0.732754,0.905929,0.782914,0.779727,0.510066,0.880614,0.745157,0.489774,0.672370,0.759146,0.826194,0.766928,0.796589,0.715603,0.730765,0.774743,0.748095,0.722675,0.411795,0.626364,0.894940,0.805261,0.847421,0.875828,0.852120,0.839547,0.842807,0.776802,0.742453,0.834237,0.792267,0.861581,0.860497,0.809104,0.607975,0.750657,0.865160,0.854839,0.812955,0.781438,0.685546,0.197357
2,ejv2xln0,0.498984,0.430952,0.379334,0.447198,0.427484,0.136433,0.525795,0.601944,0.350439,0.645880,0.500992,0.495980,0.228100,0.563923,0.411030,0.149464,0.331208,0.423315,0.560627,0.359007,0.546911,0.431786,0.356227,0.487584,0.446145,0.463828,0.141113,0.221931,0.538692,0.459141,0.584447,0.601795,0.472295,0.405798,0.405580,0.342915,0.360031,0.467659,0.422727,0.549793,0.564138,0.395977,0.373270,0.403990,0.561510,0.568355,0.578534,0.480300,0.453456,0.045800
3,2b73a28n,0.722706,0.707721,0.509893,0.688653,0.673477,0.460392,0.736154,0.745663,0.583307,0.779096,0.674365,0.733126,0.546839,0.776779,0.702145,0.341471,0.475131,0.585146,0.708204,0.621153,0.713491,0.619412,0.584529,0.538679,0.680818,0.644221,0.365278,0.575426,0.735927,0.618939,0.776189,0.664323,0.772608,0.686741,0.745538,0.639971,0.702932,0.727490,0.653448,0.720112,0.756367,0.659465,0.524263,0.685404,0.777640,0.704267,0.763601,0.658516,0.566715,0.165385
4,9785vg6d,0.784565,0.694703,0.585636,0.714717,0.654176,0.524959,0.850373,0.882554,0.673269,0.887617,0.777947,0.795195,0.486774,0.838309,0.701806,0.411637,0.577186,0.661345,0.872198,0.712044,0.794241,0.699666,0.624473,0.697629,0.681121,0.718872,0.346116,0.535897,0.822029,0.749277,0.837317,0.810106,0.834032,0.805102,0.733642,0.699602,0.772806,0.771533,0.742790,0.803458,0.883201,0.785348,0.610024,0.748325,0.855124,0.861329,0.843592,0.708874,0.682017,0.218055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191170,z4ro6lmh,0.794034,0.624479,0.509463,0.753063,0.609186,0.574060,0.764455,0.745514,0.739772,0.755611,0.731637,0.666584,0.439192,0.739244,0.693851,0.528724,0.682815,0.658582,0.701900,0.702433,0.707321,0.635850,0.695226,0.724759,0.671555,0.630638,0.262875,0.616960,0.790911,0.748170,0.725088,0.722748,0.775518,0.754473,0.649815,0.735995,0.722540,0.736884,0.740395,0.723317,0.798859,0.773625,0.537622,0.677370,0.779874,0.753726,0.729876,0.688082,0.566610,0.218233
191171,hi8k8wvb,0.705673,0.617568,0.436568,0.574614,0.597392,0.461285,0.636284,0.765217,0.523858,0.673549,0.605334,0.543787,0.543372,0.617818,0.506871,0.369045,0.553222,0.599898,0.643053,0.466036,0.677955,0.710215,0.609389,0.519294,0.639241,0.661586,0.201240,0.495555,0.726867,0.643732,0.759870,0.715031,0.699513,0.639277,0.661385,0.547860,0.497357,0.600634,0.643973,0.637813,0.746112,0.609898,0.575532,0.639404,0.694279,0.719769,0.687152,0.663382,0.541657,0.224542
191172,ma3ndg41,0.850092,0.799343,0.616650,0.725864,0.754112,0.663770,0.827476,0.820969,0.716526,0.812072,0.809880,0.774469,0.561903,0.761202,0.731910,0.501808,0.623038,0.738016,0.778583,0.745208,0.773849,0.641685,0.718069,0.660087,0.796883,0.809641,0.417493,0.587168,0.852018,0.784274,0.776513,0.772021,0.820081,0.841753,0.727239,0.757053,0.789810,0.825589,0.817259,0.784786,0.852513,0.769411,0.693576,0.7

In [49]:
c = all_sims.melt(
    id_vars=['doc_id'],
    var_name='topic',
    value_name='similarity',
)

c.columns = ['document_id', 'topic', 'similarity']
new_results = pd.merge(c, df_relevances, left_on=['document_id', 'topic'], right_on=['document_id', 'topic'])
del new_results['judgement']
nae = np.array(new_results)
results = []
for i, row in enumerate(nae):
  results.append(f'{row[1]} 0 {row[0]} 0 {row[2]} 0')

In [50]:
# Write results to file
with open('results_gensim_20ep.txt', 'w') as f:
    f.write('\n'.join(results))

In [52]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

print("Sentence embeddings:")
print(sentence_embeddings)

100%|██████████| 405M/405M [00:26<00:00, 15.5MB/s]


Sentence embeddings:
[[-0.10409425  0.5274767   1.1797731  ... -0.43389115 -0.6945233
   0.5386927 ]
 [-0.13118456 -0.17390285  1.1052188  ...  0.02624461 -0.00269846
   0.916111  ]
 [-0.7489929   0.71891737 -1.039457   ...  0.15582633  1.0202514
   0.09790451]]
